In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
half = sym.Rational(1,2)
main_panels = 20
p = RollRig.base_params(Dofs=1, panels=main_panels)

In [3]:
# Generate Referecne Frame
wing_frame = mf.HomogenousTransform().R_x(sym.pi+p.q[0])#.R_y(p.alpha_r)

#Generate Mass Matrices
M_w = ele.MassMatrix(p.m_w)
I_w = ele.MassMatrix(0,I_xx = p.I_xx_w)

# Generate Rigid Elements
Wing_point_mass = ele.RigidElement(wing_frame.Translate(0,p.y_w,p.z_w),M_w,gravityPotential=True)
wing_inertia = ele.RigidElement(wing_frame,I_w)

In [4]:
# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce_Inverted.PerUnitSpan(p,wing_frame.Translate(0,p.y_n,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = 0,
                               alpha_zero = p.alpha_c,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True)#.integrate((p.y_w,-p.s_w*half,p.s_w*half))

In [5]:
# split Main Wing into segments
forces = []
for i in range(main_panels):
    seg_width = p.s_w/main_panels
    yi = -p.s_w/2 + seg_width/2 + i*seg_width
    print(yi)
    subs = {p.y_n:yi,p.a_0:p.a[i],p.beta:-p.beta} if i < main_panels/2 else {p.y_n:yi,p.a_0:p.a[i]}
    forces.append(wing_AeroForces.subs({p.y_n:yi,p.a_0:p.a[i]})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
wing_AeroForces = ef.ExternalForce(Q)

-19*s*(1 - sigma)/40
-17*s*(1 - sigma)/40
-3*s*(1 - sigma)/8
-13*s*(1 - sigma)/40
-11*s*(1 - sigma)/40
-9*s*(1 - sigma)/40
-7*s*(1 - sigma)/40
-s*(1 - sigma)/8
-3*s*(1 - sigma)/40
-s*(1 - sigma)/40
s*(1 - sigma)/40
3*s*(1 - sigma)/40
s*(1 - sigma)/8
7*s*(1 - sigma)/40
9*s*(1 - sigma)/40
11*s*(1 - sigma)/40
13*s*(1 - sigma)/40
3*s*(1 - sigma)/8
17*s*(1 - sigma)/40
19*s*(1 - sigma)/40


In [6]:
Forcing = ef.CustomForce(None) 

CompositeForce = ef.CompositeForce([wing_AeroForces,Forcing])
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[Wing_point_mass,wing_inertia],ExtForces = CompositeForce)

In [7]:
sm.to_file('RollRigModel-Fixed.py')

In [8]:
sm.ExtForces.Q()

⎡    2      2           2 ⎛      19⋅s⋅(1 - σ)⋅q₀̇⎞       2      2           2 
⎢19⋅V ⋅c⋅ρ⋅s ⋅a₀⋅(1 - σ) ⋅⎜α_c - ───────────────⎟   17⋅V ⋅c⋅ρ⋅s ⋅a₁⋅(1 - σ) ⋅⎜
⎢                         ⎝            40⋅V     ⎠                            ⎝
⎢──────────────────────────────────────────────── + ──────────────────────────
⎣                      1600                                               1600

⎛      17⋅s⋅(1 - σ)⋅q₀̇⎞    2      2            2 ⎛      s⋅(1 - σ)⋅q₀̇⎞      2
α_c - ───────────────⎟   V ⋅c⋅ρ⋅s ⋅a₁₀⋅(1 - σ) ⋅⎜α_c + ────────────⎟   3⋅V ⋅c⋅
            40⋅V     ⎠                          ⎝          40⋅V    ⎠          
────────────────────── - ─────────────────────────────────────────── - ───────
                                             1600                             

      2            2 ⎛      3⋅s⋅(1 - σ)⋅q₀̇⎞    2      2            2 ⎛      s
ρ⋅s ⋅a₁₁⋅(1 - σ) ⋅⎜α_c + ──────────────⎟   V ⋅c⋅ρ⋅s ⋅a₁₂⋅(1 - σ) ⋅⎜α_c + ─────
                  ⎝           40⋅V     ⎠          

In [9]:
test = wing_frame.Translate(0,p.s/2,0)

In [10]:
test.BodyJacobian(p.q)

⎡                                                                             
⎢                                                                             
⎢                          ⎛            ⎛       2            2    ⎞        ⎞  
⎢                          ⎜s⋅sin(q₀)   ⎜  s⋅sin (q₀)   s⋅cos (q₀)⎟        ⎟  
⎢                        - ⎜───────── + ⎜- ────────── - ──────────⎟⋅sin(q₀)⎟⋅c
⎢                          ⎝    2       ⎝      2            2     ⎠        ⎠  
⎢                                                                             
⎢⎛            ⎛       2            2    ⎞        ⎞           ⎛     2          
⎢⎜s⋅sin(q₀)   ⎜  s⋅sin (q₀)   s⋅cos (q₀)⎟        ⎟           ⎜s⋅sin (q₀)   s⋅c
⎢⎜───────── + ⎜- ────────── - ──────────⎟⋅sin(q₀)⎟⋅sin(q₀) + ⎜────────── + ───
⎢⎝    2       ⎝      2            2     ⎠        ⎠           ⎝    2           
⎢                                                                             
⎢                                                   

In [11]:
sym.trigsimp(test.BodyJacobian(p.q))[2]